In [1]:
from pandas.core.indexes.base import Index
import requests
from lxml import etree
import pandas as pd
from datetime import datetime
import time
import os
from IPython.display import display

In [2]:
def TaiwanIndex():

    #建構爬取網站及參數
    url = 'https://www.taifex.com.tw/cht/5/indexMarging'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }
    data = {
        'origin': 'https://www.taifex.com.tw',
        'referer': 'https://www.taifex.com.tw/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
        'sec-ch-ua-platform': "Windows",
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site'
        }

    res = requests.post(url=url, headers=header, data=data)
    res.encoding='utf-8'

    #建立股價指數類表格
    TW_Index = pd.DataFrame(columns=['代碼','商品別','結算保證金','維持保證金','原始保證金'])
    name_id = []
    clearing = []
    maintainance = []
    initial = []

    #用etree爬取商品別及保證金
    tree = etree.HTML(res.text)
    path = tree.xpath('/html//div[@name="printhere"]//tr')

    #爬取後存入list
    for futures in path[1:len(path)]: 
        td = futures.xpath('./td') 
        temp_name_id = td[0].text 
        temp_clearing = td[1].text
        temp_maintainance = td[2].text
        temp_initial = td[3].text
        
        name_id.append(temp_name_id)
        clearing.append(temp_clearing)
        maintainance.append(temp_maintainance)
        initial.append(temp_initial)

    #期貨代碼list
    code_list = ['TX','MTX','*****','*****','*****','TE','ZEF','*****','*****','TF','ZFF','*****','*****','T5F','XIF','GTF','G2F','E4F','BTF','TJF','UDF','SPF','UNF','F1F']

    #將list寫入TW_Index
    TW_Index['商品別'] = name_id
    TW_Index['結算保證金'] = clearing
    TW_Index['維持保證金'] = maintainance
    TW_Index['原始保證金'] = initial
    TW_Index['代碼'] = code_list

    #找到含有TW_Index的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if 'TW_Index' in file:
                compare_file = os.path.join(filepath,file)
    
    #開啟之前爬蟲的檔案進行比較
    compare_db = pd.read_excel(compare_file)
    result = TW_Index.merge(compare_db, how='outer', indicator=True).loc[lambda x : x['_merge'] == 'left_only']
 
    if result.empty:
        print("******************************************"*2)
        print("TW_Index的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_TW_Index.xlsx".format(dte)
        
        with pd.ExcelWriter(fname) as writer:
            TW_Index.to_excel(writer, sheet_name="TW_Index")
        
        print("******************************************"*2)
        display(result)
        print("******************************************"*2)
        print("TW_Index已下載最新資料，需要更新保證金喔！")

In [3]:
def TaiwanStock ():

    #建構爬取網站及參數
    url = 'https://www.taifex.com.tw/cht/5/stockMargining'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }

    res = requests.post(url=url, headers=header)
    res.encoding='utf-8'

    #建立股票類表格
    TW_Stock = pd.DataFrame(columns=['股票期貨英文代碼','股票期貨標的證券代號','股票期貨中文簡稱','股票期貨標的證券','保證金所屬級距','結算保證金適用比例','維持保證金適用比例','原始保證金適用比例'], dtype='str')
    en_code = []
    tw_code = []
    tw_name = []
    tw_full_name = []
    margin_level = []
    clearing_ptg = []
    maintainance_ptg = []
    initial_ptg = []

    #用etree爬取商品別及保證金
    tree = etree.HTML(res.text)
    path = tree.xpath('/html/body/div[1]/div[4]/div[2]/div/div[2]/div[1]/table[1]/tbody/tr')
        
    #爬取後存入list
    for futures in path[1:len(path)]: 
        td = futures.xpath('.//td') 
        temp_en_code = td[1].text.replace(" ", "")
        temp_tw_code = td[2].text.replace(" ", "")
        temp_tw_name = td[3].text.replace(" ", "")
        temp_tw_full_name = td[4].text.replace(" ", "")
        temp_margin_level = td[5].text.replace("\r\n                ", "")
        temp_clearing_ptg = td[6].text.replace(" ", "")
        temp_maintainance_ptg = td[7].text.replace(" ", "")
        temp_initial_ptg = td[8].text.replace(" ", "")
        
        en_code.append(temp_en_code)
        tw_code.append(temp_tw_code)
        tw_name.append(temp_tw_name)
        tw_full_name.append(temp_tw_full_name)
        margin_level.append(temp_margin_level)
        clearing_ptg.append(temp_clearing_ptg)
        maintainance_ptg.append(temp_maintainance_ptg)
        initial_ptg.append(temp_initial_ptg)

    #將list寫入TW_Index
    TW_Stock['股票期貨英文代碼'] = en_code
    TW_Stock['股票期貨標的證券代號'] = tw_code
    TW_Stock['股票期貨中文簡稱'] = tw_name
    TW_Stock['股票期貨標的證券'] = tw_full_name
    TW_Stock['保證金所屬級距'] = margin_level
    TW_Stock['結算保證金適用比例'] = clearing_ptg
    TW_Stock['維持保證金適用比例'] = maintainance_ptg
    TW_Stock['原始保證金適用比例'] = initial_ptg
    TW_Stock.loc[TW_Stock['保證金所屬級距'] == '', '保證金所屬級距'] = '*****' #把空白取代成*****

    #找到含有TW_Stock的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if 'TW_Stock' in file:
                compare_file = os.path.join(filepath,file)

    #開啟之前爬蟲的檔案進行比較
    compare_db = pd.read_excel(compare_file, dtype='str')
    compare_db.loc[compare_db['保證金所屬級距'] == '', '保證金所屬級距'] = '*****' #把空白取代成*****
    result = TW_Stock.merge(compare_db, how='outer', indicator=True).loc[lambda x : x['_merge'] == 'left_only']
    
    if result.empty:
        print("******************************************"*2)
        print("TW_Stock的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_TW_Stock.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            TW_Stock.to_excel(writer, sheet_name="TW_Stock")
        
        display(result)
        print("******************************************"*2)
        print("TW_Stock已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)

In [4]:
def TaiwanETF():

    #建構爬取網站及參數
    url = 'https://www.taifex.com.tw/cht/5/stockMargining'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }

    res = requests.post(url=url, headers=header)
    res.encoding='utf-8'

    #建立ETF類表格
    TW_ETF = pd.DataFrame(columns=['股票期貨英文代碼','股票期貨標的證券代號','股票期貨中文簡稱','股票期貨標的證券','結算保證金','維持保證金','原始保證金'], dtype='str')
    en_code = []
    tw_code = []
    tw_name = []
    tw_full_name = []
    clearing = []
    maintainance = []
    initial = []

    #用etree爬取商品別及保證金
    tree = etree.HTML(res.text)
    path = tree.xpath('/html/body/div[1]/div[4]/div[2]/div/div[2]/div[1]/table[2]/tbody/tr')

    for futures in path[1:len(path)]:
        td = futures.xpath('.//td')
        temp_en_code = td[1].text.replace(" ", "")
        temp_tw_code = td[2].text.replace(" ", "")
        temp_tw_name = td[3].text.replace(" ", "")
        temp_tw_full_name = td[4].text.replace(" ", "")
        temp_clearing = td[5].text.replace(" ", "")
        temp_maintainance = td[6].text.replace(" ", "")
        temp_initial = td[7].text.replace(" ", "")
        
        en_code.append(temp_en_code)
        tw_code.append(temp_tw_code)
        tw_name.append(temp_tw_name)
        tw_full_name.append(temp_tw_full_name)
        clearing.append(temp_clearing)
        maintainance.append(temp_maintainance)
        initial.append(temp_initial)

    #將list寫入TW_Index
    TW_ETF['股票期貨英文代碼'] = en_code
    TW_ETF['股票期貨標的證券代號'] = tw_code
    TW_ETF['股票期貨中文簡稱'] = tw_name
    TW_ETF['股票期貨標的證券'] = tw_full_name
    TW_ETF['結算保證金'] = clearing
    TW_ETF['維持保證金'] = maintainance
    TW_ETF['原始保證金'] = initial

    #找到含有TW_Stock的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if 'TW_ETF' in file:
                compare_file = os.path.join(filepath,file)

    #開啟之前爬蟲的檔案進行比較
    compare_db = pd.read_excel(compare_file, dtype='str')
    result = TW_ETF.merge(compare_db, how='outer', indicator=True).loc[lambda x : x['_merge'] == 'left_only']
    
    if result.empty:
        print("******************************************"*2)
        print("TW_ETF的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_TW_ETF.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            TW_ETF.to_excel(writer, sheet_name="TW_ETF")
        
        display(result)
        print("******************************************"*2)
        print("TW_Stock已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)

In [5]:
if __name__ == '__main__' :
    
    start = time.time()
    
    TaiwanIndex()
    TaiwanStock()
    TaiwanETF()

    time_consumed = time.time() - start
    print("「總共耗時："+str(time_consumed)+" 秒」")

************************************************************************************
TW_Index的資料已是最新狀態囉！
************************************************************************************
************************************************************************************
TW_Stock的資料已是最新狀態囉！
************************************************************************************
************************************************************************************
TW_ETF的資料已是最新狀態囉！
************************************************************************************
「總共耗時：14.459307670593262 秒」
